In [23]:
import importlib
from pyvis.network import Network

In [83]:
import parse_utils
importlib.reload(parse_utils)
from parse_utils import *

In [84]:
f = open("data/sentence.txt")
sentence = f.readlines()[0]

In [85]:
doc = nlp(sentence)

In [86]:
keyphrases = get_rule_based_keyphrases(doc)
for kp in keyphrases:
    print(kp)

retail digital payments
Indian Payments Handbook
steady pace
UPI transactions
PwC report
day
percent
country


In [87]:
all_verbs = []
root_verb = ""
dep_struct = dep_tree_structure(doc)
for item in dep_struct:
    pos = item[1]
    word = item[-1]
    if pos == "VERB":
        all_verbs.append(word)
        if item[2] == "ROOT":
            root_verb = word
for v in all_verbs:
    if v == root_verb:
        print(v,"---- ROOT")
    else:
        print(v)

Growing
reach
accounting
said ---- ROOT
titled
titled


In [88]:
main_graph = make_main_graph(dep_struct)
print("Main dependency graph ready")
nt = Network(directed=True)
nt.from_nx(main_graph)
nt.show("main_graph.html",notebook=False )

Main dependency graph ready
main_graph.html


In [78]:
for vrb in all_verbs:
    if vrb != root_verb:
        path = find_path_between_verbs(vrb,root_verb,main_graph)
        path_string = [path[0]]
        related_phrases = []
        for p in path[1:]:
            for kp in keyphrases:
                if p in kp:
                    path_string.append(kp)
        path_string.append(path[-1])
        path_string = " ".join(path_string)
        print(path_string)

According stood


In [79]:
# kp = "EU foreign policy chief"
relabel = {}
drop_nodes = []
for kp in keyphrases:
    merge = []
    all_words = kp.split()
    for wd in all_words:
        neighbours = main_graph.neighbors(wd)
        flag = True
        for n in neighbours:
            if n not in all_words:
                flag = False
        if flag:
            merge.append(wd)
    # print(merge)
    representative = list(set(all_words).difference(set(merge)))
    drop_nodes.extend(merge)
    # print(representative)
    for r in representative:
        relabel.update({r:kp})
for k in relabel.keys():
    print(k,"REPLACE BY",relabel[k])

report REPLACE BY May annual report
data REPLACE BY UPI transaction data
UPI REPLACE BY aggregate UPI
Corp. REPLACE BY National Payments Corp.
Bank REPLACE BY Reserve Bank
aggregate REPLACE BY aggregate UPI
India REPLACE BY India
RBI REPLACE BY RBI
NPCI REPLACE BY NPCI
transactions REPLACE BY transactions
FY23 REPLACE BY FY23


In [80]:
all_nodes = list(main_graph.nodes)
for a_node in all_nodes:
    if a_node in stopwords:
        in_deg = main_graph.in_degree(a_node)
        out_deg = main_graph.out_degree(a_node)
        if in_deg == 0:
            drop_nodes.append(a_node)           

In [81]:
print(drop_nodes)

['May', 'annual', 'transaction', 'National', 'Payments', 'Reserve', 'the', 'and']


In [82]:
temp_graph = main_graph.copy()
temp_graph.remove_nodes_from(drop_nodes)
merged_graph = nx.relabel_nodes(temp_graph,relabel)
nt = Network(directed=True)
nt.from_nx(merged_graph)
nt.show("merged_graph.html",notebook=False )

merged_graph.html
